In [ ]:
import numpy as np
# import pandas as pd
# import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
%matplotlib inline
plt.style.use('bmh')
import tensorflow as tf
from tensorflow.keras import datasets, layers, models